In [30]:
import pandas as pd
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
from torch.autograd import Variable
import os 


def Normlize(Z):
    Zmax, Zmin = Z.max(axis=1), Z.min(axis=1)
    Zmean = Z.mean(axis=1)
    #按列排序
    Zmax, Zmin = Zmax.reshape(-1, 1), Zmin.reshape(-1, 1)
    Zmean = Zmean.reshape(-1, 1)
    Z = (Z - Zmean) / (Zmax - Zmin)
    return Z


def Data_Reading(Normalization=True):
    train_x = np.load('F:\\gitworkspace\\python\\e-nose-cnn\\train\\trainset.npy')
    train_y = np.load('F:\\gitworkspace\\python\\e-nose-cnn\\train\\trainlabel.npy')

    test_x = np.load('F:\\gitworkspace\\python\\e-nose-cnn\\test\\testset.npy')
    test_y = np.load('F:\\gitworkspace\\python\\e-nose-cnn\\test\\testlabel.npy')

    # Normalization
    train_x_Normed = Normlize(train_x)

    test_x_Normed = Normlize(test_x)

    # xlsx to tensor
    if Normalization:
        train_x = torch.from_numpy(train_x_Normed).type(torch.cuda.FloatTensor)

        test_x = torch.from_numpy(test_x_Normed).type(torch.cuda.FloatTensor)

        train_y = torch.from_numpy(train_y).type(torch.int64)

        test_y = torch.from_numpy(test_y).type(torch.int64)

    else:
        train_x = torch.from_numpy(train_x).type(torch.FloatTensor)

        test_x = torch.from_numpy(test_x).type(torch.FloatTensor)

        train_y = torch.from_numpy(train_y).type(torch.int64)

        test_y = torch.from_numpy(test_y).type(torch.int64)
    # reshape
    train_x = train_x.view(408, 1, 10, 120)
    test_x = test_x.view(202, 1, 10, 120)

    return train_x, test_x, train_y, test_y


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # self.conv1 = nn.Conv2d(1,6,(4,4),stride=(2,2))#(10 - 4)/2 + 1 = 4   (120 - 4)/2 + 1 = 59 
        # self.conv2 = nn.Conv2d(6,10,(3,3),stride=(1,2))#(4 - 3)/1 + 1 = 2   (59 - 3)/2 + 1 = 29 
        # self.conv3 = nn.Conv2d(10,14,(2,2),stride=(1,1))#(2 - 2)/1 + 1 = 1  (29 - 2)/1 + 1 = 28
        self.conv1 = nn.Conv2d(1,6,(10,3),stride=(1,3))#40
        self.conv2 = nn.Conv2d(6,10,(1,4),stride=(1,2))#19
        self.fc = nn.Linear(10*1*19,4)
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.xavier_uniform_(m.weight)
                init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                init.normal_(m.weight, std=0.01)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        # x = F.relu(self.conv3(x))
        x = x.view(x.size(0), -1)
        # return x
        return F.log_softmax(x,dim=0)


#training
batch_size = 17
def train(train_x,train_y,step=10):
    for epoch in range(200):
        running_loss = 0.0
        for i in range(0,(int)(len(train_x)/batch_size)):
            t_x = Variable(train_x[i*batch_size:i*batch_size+batch_size])
            t_y = Variable(train_y[i*batch_size:i*batch_size+batch_size])
            out = cnn(t_x)
            #forward
            loss = loss_func(out, t_y)
            #梯度初始化为零
            optimizer.zero_grad()
             
            #backward
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
   
        running_loss = running_loss / 24

        if (epoch + 1) % step == 0:
            print('Epoch[{}], loss: {:.8f}'.format(epoch + 1, running_loss))
        

def classification(test_x,test_y):
    sum = 0

    te_x = Variable(test_x)
    tey1 = Variable(test_y)
    out1 = cnn(te_x)
    predicted1 = torch.max(out1.data, 1)[1].data.squeeze()
    total = tey1.size(0)

    for j in range(tey1.size(0)):
        if predicted1[j] == tey1[j]:
            sum += 1

    print('out:{}, total:{}, accuracy:{}, sum:{}'.format(predicted1, total, sum / total, sum))


if __name__ == '__main__':
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")#cuda:0

    print(device)

    cnn = Net()
    print(cnn)
    cnn.to(device)

    #sgd -> stochastic gradient descent
    optimizer = optim.SGD(cnn.parameters(), lr=0.0001, momentum=0.8)
    loss_func = nn.CrossEntropyLoss()

    train_x, test_x, train_y, test_y = Data_Reading(Normalization=1)
    train_y = train_y.squeeze()
    test_y = test_y.squeeze()
    train_x = train_x.to(device)
    test_x = test_x.to(device)
    train_y = train_y.to(device)
    test_y = test_y.to(device)

    train(train_x, train_y, step=10)
    classification(test_x,test_y)

cuda:0
Net(
  (conv1): Conv2d(1, 6, kernel_size=(10, 3), stride=(1, 3))
  (conv2): Conv2d(6, 10, kernel_size=(1, 4), stride=(1, 2))
  (fc): Linear(in_features=190, out_features=4, bias=True)
)
Epoch[10], loss: 5.24723669
Epoch[20], loss: 5.24723323
Epoch[30], loss: 5.24722970
Epoch[40], loss: 5.24722606
Epoch[50], loss: 5.24722282
Epoch[60], loss: 5.24721968
Epoch[70], loss: 5.24721658
Epoch[80], loss: 5.24721368
Epoch[90], loss: 5.24721054
Epoch[100], loss: 5.24720750
Epoch[110], loss: 5.24720448
Epoch[120], loss: 5.24720122
Epoch[130], loss: 5.24719799
Epoch[140], loss: 5.24719477
Epoch[150], loss: 5.24719153
Epoch[160], loss: 5.24718833
Epoch[170], loss: 5.24718503
Epoch[180], loss: 5.24718171
Epoch[190], loss: 5.24717842
Epoch[200], loss: 5.24717478
out:tensor([189,  18, 152, 152, 189,  18, 189, 189, 152,  18,  18,  18,  18,  18,
        152, 189, 189,  18, 188,  18, 152, 152, 152, 183, 152,  18, 152,  18,
        185, 185, 187, 152, 152, 152, 152, 152, 152, 152, 152,   6, 177, 152